In [1]:
import os
from dotenv import load_dotenv
from autogen import AssistantAgent

load_dotenv()

llm_config = {"model": "gpt-4o-mini"}

project_manager = AssistantAgent(
    name="project_manager",
    llm_config=llm_config,
    system_message=(
        "You are a project manager. Your task is to receive the customer's request, "
        "and send a precise and detailed message to each of your engineers to get an estimate. "
        "You will talk to them one by one."
    )
)

requirement_engineer = AssistantAgent(
    name="requirement_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a requirement engineer. Based on the customer's request and historical data from the project manager, "
        "your job is to create an estimate for the requirement tasks. "
        "Your response must be ONLY the table-based estimate in Markdown format, following the structure provided. "
        "Do not add any extra text, explanation, or conversational filler."
    )
)

system_engineer = AssistantAgent(
    name="system_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a system engineer. Based on the customer's request and historical data from the project manager, "
        "your job is to create an estimate for the design tasks. "
        "Your response must be ONLY the table-based estimate in Markdown format, following the structure provided. "
        "Do not add any extra text, explanation, or conversational filler."
    )
)

software_engineer = AssistantAgent(
    name="software_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a software engineer. Based on the customer's request and historical data from the project manager, "
        "your job is to create an estimate for the programming tasks. "
        "Your response must be ONLY the table-based estimate in Markdown format, following the structure provided. "
        "Do not add any extra text, explanation, or conversational filler."
    )
)

testing_engineer = AssistantAgent(
    name="testing_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a testing engineer. Based on the customer's request and historical data from the project manager, "
        "your job is to create an estimate for the testing tasks. "
        "Your response must be ONLY the table-based estimate in Markdown format, following the structure provided. "
        "Do not add any extra text, explanation, or conversational filler."
    )
)

documentation_engineer = AssistantAgent(
    name="documentation_engineer",
    llm_config=llm_config,
    system_message=(
        "You are a documentation engineer. Based on the customer's request and historical data from the project manager, "
        "your job is to create an estimate for the documentation tasks. "
        "Your response must be ONLY the table-based estimate in Markdown format, following the structure provided. "
        "Do not add any extra text, explanation, or conversational filler."
    )
)

customer_request = """
I want to create a web-based mobile app for my insurance company where my customers can input their budget about the insurance plan,
the reason why they need the insurance, and their status now (e.g., student, adult).
After the app receives all the information, we can use it to recommend the best insurance plan and explain why we made that recommendation.
"""

req_eng_prompt = f"""
Based on the following customer request:
---
{customer_request}
---
Please follow your workflow (Write Requirement -> Review Requirements -> Rework) and use the quality record format I provide here to give me the estimated Requirement task plan.

quality record for requirement task
HW1
                                  Amount of work      Productivity Rate
Write requirements                156 req             5 req/hour
Review requirement
    Preparation for review        156 req             8 req/hour
    Review Meeting                156 req             8 req/hour
Rework                            97 defects          5 defects/hour
HW2
                                  Amount of work      Productivity Rate
Write requirements                165 req             4 req/hour
Review requirement
    Preparation for review        165 req             5 req/hour
    Review Meeting                165 req             10 req/hour
Rework                            210 defects         8 defects/hour
"""

sys_eng_prompt = f"""
Based on the following customer request:
---
{customer_request}
---
Please follow your workflow (Write DD -> Review DD-> Rework) and use the quality record format I provide here to give me the estimated Design task plan.

quality record for design task
HW1
                                  Amount of work      Productivity Rate
Write Design Document (DD)        134 pages           5 pages/hour
Review DD
    Preparation for DD            134 pages           3 pages/hour
    Review DD                     134 pages           6 pages/hour
Rework                            88 defects          5 defects/hour
HW2
                                  Amount of work      Productivity Rate
Write Design Document (DD)        10 pages            6 pages/hour
Review DD
    Preparation for DD            10 pages            5 pages/hour
    Review DD                     10 pages            10 pages/hour
Rework                            278 defects         10 defects/hour
"""

sw_eng_prompt = f"""
Based on the following customer request:
---
{customer_request}
---
Please follow your workflow (Write Code -> Unit Testing -> Code Inspection -> Rework) and use the quality record format I provide here to give me the estimated programming task plan.

quality record programming task
HW1
                                  Amount of work      Productivity Rate
Write code                        3945 SLOC           5 SLOC/hour
Unit Testing
    Prepare/Execute Test Cases    187 test case       3 test case/hour
    Fix Found Defects             155 defects         5 defects/day
    Test Fixed Defects            155 defects         10 defects/day
Code Inspection
    Preparation for review        3945 SLOC           125 SLOC/hour
    Review Meeting                3945 SLOC           200 SLOC/hour
Rework                            314 defects         3 defects/hour
HW2
                                  Amount of work      Productivity Rate
Write code                        3450 SLOC           10 SLOC/hour
Unit Testing
    Prepare/Execute Test Cases    174 test case       5 test case/hour
    Fix Found Defects             104 defects         4 defects/day
    Test Fixed Defects            104 defects         5 defects/day
Code Inspection
    Preparation for review        3450 SLOC           110 SLOC/hour
    Review Meeting                3450 SLOC           160 SLOC/hour
Rework                            309 defects         5 defects/hour
"""

test_eng_prompt = f"""
Based on the following customer request:
---
{customer_request}
---
Please follow your workflow (Write test plan(TP) -> Review TP -> Execute TP(test case) -> Fix Found Defects) and use the quality record format I provide here to give me the estimated Testing task plan.

quality record for testing task
HW1
                                  Amount of work      Productivity Rate
Write test plan (TP)              203 pages           6 pages/day
Review TP
    Preparation for TP            203 pages           2 pages/hour
    Review TP Meeting             203 pages           6 pages/hour
    Rework                        102 defects         5 defects/day
Execute TP (test cases)           310 test case       6 test case/day
Fix Found Defects                 185 defects         12 defects/day
HW2
                                  Amount of work      Productivity Rate
Write test plan (TP)              149 pages           5 pages/day
Review TP
    Preparation for TP            149 pages           5 pages/hour
    Review TP Meeting             149 pages           6 pages/hour
    Rework                        99 defects          4 defects/day
Execute TP (test cases)           138 test case       30 test case/day
Fix Found Defects                 77 defects          4 defects/day
"""

doc_eng_prompt = f"""
Based on the following customer request:
---
{customer_request}
---
Please follow your workflow (User Documentation (UD) -> Review UD -> Rework) and use the quality record format I provide here to give me the estimated Documentation task plan.

quality record documentation task
HW1
                                  Amount of work      Productivity Rate
User Documentation                141 pages           5 pages/hour
Review UD
    Preparation for review        141 pages           4 pages/hour
    Review Meeting                141 pages           10 pages/hour
Rework                            203 defects         10 defects/hour
HW2
                                  Amount of work      Productivity Rate
User Documentation                195 pages           4 pages/hour
Review UD
    Preparation for review        195 pages           5 pages/hour
    Review Meeting                195 pages           8 pages/hour
Rework                            344 defects         15 defects/hour
"""

estimates = {}

print("--- Getting Estimate from Requirement Engineer ---")
project_manager.initiate_chat(
    recipient=requirement_engineer,
    message=req_eng_prompt,
    max_turns=1,
    summary_method="last_msg"
)
estimates["requirement"] = requirement_engineer.last_message()["content"]

print("\n--- Getting Estimate from System Engineer ---")
project_manager.initiate_chat(
    recipient=system_engineer,
    message=sys_eng_prompt,
    max_turns=1,
    summary_method="last_msg"
)
estimates["system"] = system_engineer.last_message()["content"]

print("\n--- Getting Estimate from Software Engineer ---")
project_manager.initiate_chat(
    recipient=software_engineer,
    message=sw_eng_prompt,
    max_turns=1,
    summary_method="last_msg"
)
estimates["software"] = software_engineer.last_message()["content"]

print("\n--- Getting Estimate from Testing Engineer ---")
project_manager.initiate_chat(
    recipient=testing_engineer,
    message=test_eng_prompt,
    max_turns=1,
    summary_method="last_msg"
)
estimates["testing"] = testing_engineer.last_message()["content"]

print("\n--- Getting Estimate from Documentation Engineer ---")
project_manager.initiate_chat(
    recipient=documentation_engineer,
    message=doc_eng_prompt,
    max_turns=1,
    summary_method="last_msg"
)
estimates["documentation"] = documentation_engineer.last_message()["content"]

print("\n\n===============================================")
print("          FINAL PROJECT ESTIMATES")
print("===============================================")

for role, estimate in estimates.items():
    print(f"\n----- {role.upper()} ESTIMATE -----")
    print(estimate)
    print("------------------------------------")


--- Getting Estimate from Requirement Engineer ---
project_manager (to requirement_engineer):


Based on the following customer request:
---

I want to create a web-based mobile app for my insurance company where my customers can input their budget about the insurance plan,
the reason why they need the insurance, and their status now (e.g., student, adult).
After the app receives all the information, we can use it to recommend the best insurance plan and explain why we made that recommendation.

---
Please follow your workflow (Write Requirement -> Review Requirements -> Rework) and use the quality record format I provide here to give me the estimated Requirement task plan.

quality record for requirement task
HW1
                                  Amount of work      Productivity Rate
Write requirements                156 req             5 req/hour
Review requirement
    Preparation for review        156 req             8 req/hour
    Review Meeting                156 req             8